# Detecção de Raças de Cachorros utilizando Redes Neurais

## Importações

### Importando biblioteca

In [94]:

from sys import platform
import os
import pandas as pd
from PIL import Image
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

### Importando Dataset

In [80]:
if platform == "linux" or platform == "linux2":
    folder_path = os.getcwd() + "/" + "low-resolution"
elif platform == "win32":
   folder_path = os.getcwd() + "\\" + "low-resolution"


list_images = []

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            
            file_path = os.path.join(subfolder_path, file)
            image_about = {}
            breed = subfolder.split("-")[-1]
            image_about["breed"] = breed
            image_about["filename"] = file
            image_about["path_file"] = file_path
            list_images.append(image_about)

df_images = pd.DataFrame.from_records(list_images)

### Pré-processamento

In [81]:
if platform == "linux" or platform == "linux2":
    os_path = "/" 
elif platform == "win32":
   os_path = "\\"

folder_path = os.getcwd() + os_path + "low-resolution"
slot_size = 20

#Retorna uma tupla com os path de cada imagem e o target randomizados
def load_path_of_dataset():
    arquivos = os.listdir(folder_path)
    subpastas = [subpasta for subpasta in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subpasta))]
    imagens_path = []
    labels = []
    for subpasta in subpastas:
        subpasta_path = os.path.join(folder_path, subpasta)
        imagens = os.listdir(subpasta_path)
        imagens_path.extend([subpasta + os_path + s for s in imagens])
        label = subpasta.split("-")[2]
        labels.extend([label for s in imagens])
    # minha_lista = list(zip(imagens_path, labels))
    # random.shuffle(minha_lista)
    return imagens_path, labels

#função responsavel em realizar a divisão em slots
#recebe como parametro a tupla da função load_path_of_dataset, e o numero da interação para carregar o dataframe
#retorna uma tupla menor, e o numero máximo de interações posiveis até o fim do dataset
def slot_loading(interacao, imagens_amostradas, imagens_labels):
    if interacao <= 0:
        raise ValueError('O valor da interação, deve ser maior que zero ! ')
    elif interacao > (interacao * len(imagens_amostradas)):
        raise ValueError('O valor da interação, é muito grande !')
    
    start_index = (interacao - 1) * slot_size
    end_index = interacao * slot_size
    if end_index > len(imagens_amostradas):
        return imagens_amostradas[start_index:], imagens_labels[start_index:]

    return imagens_amostradas[start_index:end_index], imagens_labels[start_index:end_index], len(imagens_amostradas) // slot_size

#Recebe como parametro uma tupla
#retorna um dataset já pré-processado e formatados
def get_images(imagens_amostradas):
    dataset = pd.DataFrame(columns=['Image'])
    img_size_height = 300
    img_size_width = 400
    for imagem_nome in imagens_amostradas:
        try:
            # extensao = os.path.splitext(imagem_nome)[1].lower()
            imagem_path = os.path.join(folder_path, imagem_nome)
            imagem = Image.open(imagem_path)
            imagem = imagem.resize((img_size_width, img_size_height))  # Redimensiona para um tamanho comum
            imagem_array = np.array(imagem) / 255.0  # Normaliza dividindo por 255
            dataset.loc[len(dataset)] = [imagem_array]
        except Exception as error:
            print("ocorreu um erro com imagem_path: ", imagem_path, " imagem_nome " + imagem_nome," imagem_nome[0] " + imagem_nome[0], " imagens_amostradas: " , imagens_amostradas)

    return dataset


#### Exemplo de utilização

In [82]:
X, y = load_path_of_dataset()
x_new, y_new, interations = slot_loading(1, X, y)
# slot_path_dataset, interations = slot_loading(1,path_dataset)
dataset = get_images(x_new);
print(dataset)

                                                Image
0   [[[0.6235294117647059, 0.6392156862745098, 0.5...
1   [[[0.5098039215686274, 0.4823529411764706, 0.6...
2   [[[0.5803921568627451, 0.23921568627450981, 0....
3   [[[0.9686274509803922, 1.0, 1.0], [0.952941176...
4   [[[0.07450980392156863, 0.08235294117647059, 0...
5   [[[0.5647058823529412, 0.5450980392156862, 0.5...
6   [[[0.615686274509804, 0.7254901960784313, 0.39...
7   [[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...
8   [[[0.4980392156862745, 0.5294117647058824, 0.4...
9   [[[0.4823529411764706, 0.5098039215686274, 0.2...
10  [[[0.7647058823529411, 0.7058823529411765, 0.5...
11  [[[0.19215686274509805, 0.21176470588235294, 0...
12  [[[0.6078431372549019, 0.596078431372549, 0.62...
13  [[[0.34509803921568627, 0.34509803921568627, 0...
14  [[[0.5882352941176471, 0.6588235294117647, 0.7...
15  [[[0.07058823529411765, 0.20784313725490197, 0...
16  [[[0.3058823529411765, 0.2784313725490196, 0.2...
17  [[[0.8392156862745098, 0

## Separação do Conjunto de teste e treino

In [92]:
# Sua tupla
X, y = load_path_of_dataset()

print(np.unique(y))


# Separando 70% para treino e 30% para teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, random_state=42)

# Exibindo as dimensões dos conjuntos de treino e teste
print("Dimensões do conjunto de treino:", len(X_treino), len(y_treino))
print("Dimensões do conjunto de teste:", len(X_teste), len(y_teste))

['Afghan_hound' 'African_hunting_dog' 'Airedale'
 'American_Staffordshire_terrier' 'Appenzeller' 'Australian_Shepherd'
 'Australian_terrier' 'Bedlington_terrier' 'Bernese_mountain_dog'
 'Bichon_Frise' 'Black_sable' 'Blenheim_spaniel' 'Border_collie'
 'Border_terrier' 'Boston_bull' 'Bouvier_des_Flandres' 'Brabancon_griffo'
 'Brittany_spaniel' 'Cane_Carso' 'Cardigan' 'Chesapeake_Bay_retriever'
 'Chihuahua' 'Chinese_Crested_Dog' 'Dandie_Dinmont' 'Doberman'
 'English_foxhound' 'English_setter' 'English_springer' 'EntleBucher'
 'Eskimo_dog' 'Fila Braziliero' 'French_bulldog' 'German_shepherd'
 'German_short_haired_pointer' 'Gordon_setter' 'Great_Dane'
 'Great_Pyrenees' 'Greater_Swiss_Mountain_dog' 'Ibizan_hound'
 'Irish_setter' 'Irish_terrier' 'Irish_water_spaniel' 'Irish_wolfhound'
 'Italian_greyhound' 'Japanese_Spitzes' 'Japanese_spaniel'
 'Kerry_blue_terrier' 'Labrador_retriever' 'Lakeland_terrier' 'Leonberg'
 'Lhasa' 'Maltese_dog' 'Mexican_hairless' 'Newfoundland' 'Norfolk_terrier'
 'No

## Criação do modelo

In [84]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
# Model Architecture
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Conv2D(filters=128 , kernel_size=2 , padding='same' , activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(133,activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 224, 224, 16)      208       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 112, 112, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 112, 112, 32)      2080      
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 56, 56, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 56, 56, 64)        8256      
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 28, 28, 64)       

#### Treinamento em batches 

In [98]:

# X = [item[0] for item in path_dataset]  # Recursos (features) da tupla
# y = [item[1] for item in path_dataset]   # Rótulos (labels) da tupla

label_encoder = LabelEncoder()
y_treino = label_encoder.fit_transform(y_treino)
y_treino = y_treino.tolist()
# Treinar o modelo
x_new, y_new, i = slot_loading(1, X_treino, y_treino)
# model.fit(x_new, y_new)

for o in range(1, i):
    print( o," of " ,i)
    x_new, y_new = None, None
    x_new, y_new, i = slot_loading(o, X_treino, y_treino)
    print(x_new)
    x_new = get_images(x_new)
    print(x_new)
    print(y_new)
    print(i)
    model.fit(x_new , y_new, 
               shuffle = True,
               batch_size = 20,
               epochs = 25,
               verbose = 1)
# # Avaliar o modelo
# score = model.score(dados_teste, rotulos_teste)
# print("Acurácia do modelo:", score)




1  of  2465
['274-n000110-Shetland_sheepdog/n136341.jpeg', '806-n000129-papillon/n170528.jpg', '3336-n000121-chinese_rural_dog/n147394.jpeg', '2909-n000116-Cardigan/n139660.jpeg', '2192-n000088-Samoyed/n125726.jpeg', '3336-n000121-chinese_rural_dog/n149939.jpeg', '3336-n000121-chinese_rural_dog/n149362.jpeg', '2192-n000088-Samoyed/n124595.jpg', '215-n000075-Chesapeake_Bay_retriever/n121448.jpg', '3580-n000122-Labrador_retriever/n151980.jpg', '207-n000011-Irish_setter/n107660.jpg', '253-n000107-Japanese_Spitzes/n133079.jpeg', '1043-n000001-Shiba_Dog/n100255.jpg', '5355-n000126-golden_retriever/n160225.jpeg', '206-n000051-Saluki/n114850.jpg', '221-n000055-EntleBucher/n117419.jpg', '2342-n000102-miniature_schnauzer/n129881.jpg', '276-n000112-English_springer/n136751.jpg', '230-n000084-Boston_bull/n123835.jpg', '211-n000059-German_short_haired_pointer/n118379.jpg']
                                                Image
0   [[[0.37254901960784315, 0.5843137254901961, 0....
1   [[[0.356862745

ValueError: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, (<class 'list'> containing values of types {"<class 'int'>"})